<span style="color: orange; font-family: Calibri Light;">
  <h1><b>MODEL EVALUATION: BERTopic (all-MiniLM-L6-v2/HDBSCAN/raw_text)</b></h1>
</span>
<span style = "font-family: Calibri Light">

model evaluation of the first bert model.

what do I need?

- the test data labelled by the bert model
- the human labels, arranged in the same order as in the test label.

for two above, since we don't have date of creation, we'll use IDs to arrange the rows

In [35]:
import pandas as pd
import os
from glob import glob
import ast
import re
import spacy
import time

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, classification_report, adjusted_rand_score
import matplotlib.pyplot as plt
import seaborn as sns

In [37]:
# Access dataset files
folder_path = os.path.join( "..", "bert_model_data") 
file_type = "*.csv"

# List of dataset file paths
document_path = glob(os.path.join(folder_path, file_type))

document_path

['../bert_model_data/bert_default_raw_text_topics_term.csv',
 '../bert_model_data/test_data_bert_raw_label.csv',
 '../bert_model_data/bert_train_lemma.csv',
 '../bert_model_data/human_and_bert_raw_labels.csv',
 '../bert_model_data/bert_default_raw_text_topics_term_labelled.csv',
 '../bert_model_data/test_data_human_label_bert_raw.csv',
 '../bert_model_data/posts_upvote_ratio.csv',
 '../bert_model_data/bert_sw_test_data.csv',
 '../bert_model_data/bert_train_no_prep.csv',
 '../bert_model_data/bert_model_1_label_train_data.csv',
 '../bert_model_data/bert-data-sw.csv',
 '../bert_model_data/test_data_lemma.csv',
 '../bert_model_data/bert_kmeans_test_data.csv',
 '../bert_model_data/bert_w2v_test_data.csv']

In [38]:
#import cleaned data

def list_converter(text):
    #to revert list->str conversion from pd.read_csv
    return ast.literal_eval(text)


model_labelled_data = pd.read_csv("../bert_model_data/test_data_bert_raw_label.csv", converters ={'tokens':list_converter})
human_labelled_data = pd.read_csv("../bert_model_data/test_data_human_label_bert_raw.csv", converters ={'tokens':list_converter})
#test_data = test_data.drop(columns = ['index', 'clean_text', 'tokens', 'word_count'])

In [39]:
model_labelled_data.drop(columns = ['Unnamed: 0'], inplace = True)
model_labelled_data.head()

,text_type,ID,date_created,long_text,topic_description
0,submission,wbwik,2012-07-10 19:11:08,Only in Dubai! R8: BRAND NEW 2012 Audi R8 V8 -...,1_dubai_uae_arabic_people
1,comment,c5c2v3q,2012-07-10 22:36:50,"Surprise! Happy birthday! \n\nAh man, *anoth...",-1_dubai_people_just_like
2,comment,c5cc9sz,2012-07-11 08:48:34,I looked at the AD and it seems more like a sa...,22_news_ad_gn_ads
3,comment,c5cdavh,2012-07-11 10:15:52,It's a marketing post. But isn't it against du...,-1_dubai_people_just_like
4,submission,10embj,2012-09-24 23:02:05,Dubai cap my dad brought me! [FYSR],1_dubai_uae_arabic_people


In [40]:
#split `topic_description` into two columns for topic number and topic description
model_labelled_data[['model_topic_no', 'model_topic_desc']] = model_labelled_data['topic_description'].str.split('_', n = 1, expand = True)
model_labelled_data['model_topic_no'] = model_labelled_data['model_topic_no'].astype(int) 
model_labelled_data.drop(columns = ['topic_description'], inplace = True)

In [41]:
#sort by `ID`
model_labelled_data.sort_values(by=['ID'], ignore_index = True, inplace = True)
model_labelled_data.head()

,text_type,ID,date_created,long_text,model_topic_no,model_topic_desc
0,submission,10embj,2012-09-24 23:02:05,Dubai cap my dad brought me! [FYSR],1,dubai_uae_arabic_people
1,submission,13ch705,2023-05-09 10:04:19,Anyone here have any idea where I can park my ...,10,lane_speed_left_parking
2,submission,13cpma4,2023-05-09 17:17:49,"UAE real estate agents and brokers, I need you...",1,dubai_uae_arabic_people
3,submission,13p1nlq,2023-05-23 00:07:39,A Well Deserved Shout-out I posted in this sub...,7,talabat_delivery_price_zomato
4,submission,14d7052,2023-06-19 10:52:36,Furnished apartments… bedding/pots and pans Hi...,24,rent_landlord_property_estate


In [42]:
model_labelled_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   text_type         1200 non-null   object
 1   ID                1200 non-null   object
 2   date_created      1200 non-null   object
 3   long_text         1200 non-null   object
 4   model_topic_no    1200 non-null   int64 
 5   model_topic_desc  1200 non-null   object
dtypes: int64(1), object(5)
memory usage: 56.4+ KB


In [43]:
#dataset showing the human labels
human_labelled_data.head()

,ID,long_text,topics,topic_number_bert_raw,topic_number_bert_w2v,topic_number_kmeans,topic_number_bert_sw
0,10embj,Dubai cap my dad brought me! [FYSR],undefined,-1,-1,-1,-1
1,13ch705,Anyone here have any idea where I can park my ...,infrastructure.transportation,32,36,20,37
2,13cpma4,"UAE real estate agents and brokers, I need you...",real_estate_housing,24,4,5,12
3,13p1nlq,A Well Deserved Shout-out I posted in this sub...,shopping,61,61,61,61
4,14d7052,Furnished apartments‚Ä¶ bedding/pots and pans ...,real_estate_housing,24,4,5,12


In [44]:
#select only columns needed for this model
human_labelled_data = human_labelled_data[
    ['ID', 'long_text', 'topics', 'topic_number_bert_raw']].copy()

In [45]:
human_labelled_data.head()

,ID,long_text,topics,topic_number_bert_raw
0,10embj,Dubai cap my dad brought me! [FYSR],undefined,-1
1,13ch705,Anyone here have any idea where I can park my ...,infrastructure.transportation,32
2,13cpma4,"UAE real estate agents and brokers, I need you...",real_estate_housing,24
3,13p1nlq,A Well Deserved Shout-out I posted in this sub...,shopping,61
4,14d7052,Furnished apartments‚Ä¶ bedding/pots and pans ...,real_estate_housing,24


In [46]:
#include column in model_labelled_data to hold the human topic numbers

model_labelled_data[['human_topic_desc','human_topic_no']] = human_labelled_data[['topics','topic_number_bert_raw']].copy()

In [47]:
model_labelled_data.head()

,text_type,ID,date_created,long_text,model_topic_no,model_topic_desc,human_topic_desc,human_topic_no
0,submission,10embj,2012-09-24 23:02:05,Dubai cap my dad brought me! [FYSR],1,dubai_uae_arabic_people,undefined,-1
1,submission,13ch705,2023-05-09 10:04:19,Anyone here have any idea where I can park my ...,10,lane_speed_left_parking,infrastructure.transportation,32
2,submission,13cpma4,2023-05-09 17:17:49,"UAE real estate agents and brokers, I need you...",1,dubai_uae_arabic_people,real_estate_housing,24
3,submission,13p1nlq,2023-05-23 00:07:39,A Well Deserved Shout-out I posted in this sub...,7,talabat_delivery_price_zomato,shopping,61
4,submission,14d7052,2023-06-19 10:52:36,Furnished apartments… bedding/pots and pans Hi...,24,rent_landlord_property_estate,real_estate_housing,24


In [48]:
model_labelled_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   text_type         1200 non-null   object
 1   ID                1200 non-null   object
 2   date_created      1200 non-null   object
 3   long_text         1200 non-null   object
 4   model_topic_no    1200 non-null   int64 
 5   model_topic_desc  1200 non-null   object
 6   human_topic_desc  1200 non-null   object
 7   human_topic_no    1200 non-null   int64 
dtypes: int64(2), object(6)
memory usage: 75.1+ KB


In [52]:
model_labelled_data.to_csv('model_evaluation/human_and_model_labels_model1.csv')

In [53]:
model_labelled_data['human_topic_desc'].unique()

array(['undefined', 'infrastructure.transportation',
       'real_estate_housing', 'shopping', 'banking_financial_services',
       'employment', 'entertainment.restrictions', 'dining_service.food',
       'employment.working_conditions', 'islam_religion',
       'social_interactions', 'supernatural_activities', 'reddit_usage',
       'taxes_levies', 'lifestyle', 'arabic_language',
       'infrastructure.communications_internet_restrictions',
       'urban_mobility.traffic', 'general_health_services',
       'entertainment', 'diversity_inclusion.race/nationality',
       'unrelated_theme', 'recreation', 'government',
       'safety.law_and_order',
       'infrastructure.mobile_communication_internet',
       'media_communication', 'personal_habits.smoking',
       'infrastructure.landmarks', 'safety.home_safety',
       'infrastructure.public_amenities', 'entertainment.football',
       'animal_welfare', 'dubai_urban_centers', 'travelling',
       'personal_grooming', 'locals_local_com

In [54]:
mask = model_labelled_data.human_topic_no ==24
model_labelled_data[mask]

,text_type,ID,date_created,long_text,model_topic_no,model_topic_desc,human_topic_desc,human_topic_no
2,submission,13cpma4,2023-05-09 17:17:49,"UAE real estate agents and brokers, I need you...",1,dubai_uae_arabic_people,real_estate_housing,24
4,submission,14d7052,2023-06-19 10:52:36,Furnished apartments… bedding/pots and pans Hi...,24,rent_landlord_property_estate,real_estate_housing,24
324,comment,dle6vky,2017-08-09 23:47:56,Yes.. my villa is haunted but by a friendly gh...,-1,dubai_people_just_like,real_estate_housing,24
634,comment,fpfvpf7,2020-05-04 13:41:27,>Real estate brokers are not all Porsche drivi...,24,rent_landlord_property_estate,real_estate_housing,24
845,comment,h8rkfha,2021-08-13 13:42:17,"Check out Barsha south, plenty of good apartme...",24,rent_landlord_property_estate,real_estate_housing,24
847,comment,h90kkqp,2021-08-15 14:14:01,"From your car to your apartment, it's more con...",-1,dubai_people_just_like,real_estate_housing,24
849,comment,h94k0m3,2021-08-16 10:19:04,Once a supermarket staff told me that I can ke...,-1,dubai_people_just_like,real_estate_housing,24
870,comment,hjjhl9r,2021-11-06 16:05:13,Purely just my opinion - brokers/landlords try...,5,expo_beach_building_khalifa,real_estate_housing,24
911,comment,hxah86c,2022-02-17 14:05:10,don't forget the:\n\nmy landlord is increasing...,24,rent_landlord_property_estate,real_estate_housing,24
1119,comment,jji0unb,2023-05-09 22:14:25,What is the rental amount you pay currently?\n...,-1,dubai_people_just_like,real_estate_housing,24


<h2> ACCURACY SCORE </h2>

<h3>Accuracy score of the entire model prediction</h3>

In [55]:
#accuracy score
y_true = model_labelled_data.human_topic_no
y_pred = model_labelled_data.model_topic_no
accuracy = accuracy_score(y_true, y_pred, normalize = True)

print (f'Bertopic model with default parameters and trained on unprocessed data\nModel accuracy is: {accuracy:.3f}')

Bertopic model with default parameters and trained on unprocessed data
Model accuracy is: 0.369


<h3>Accuracy score of the individual human labels</h3>

In [56]:
topic_range = sorted(model_labelled_data['human_topic_no'].unique())
accuracy = {'topic_no':[],
            'human_topic_desc':[],
           'accuracy_score':[]
           }

for topic_no in topic_range:
    accuracy['topic_no'].append(topic_no) 
    mask = model_labelled_data.human_topic_no == topic_no
    topic_df = model_labelled_data[mask]
    
    accuracy['human_topic_desc'].append(topic_df['human_topic_desc'].unique())
    
    #accuracy
    y_true = topic_df.human_topic_no
    y_pred = topic_df.model_topic_no
    acc = accuracy_score(y_true, y_pred, normalize = True)
    
    accuracy['accuracy_score'].append(acc)

In [57]:
accuracy_df = pd.DataFrame(accuracy)
accuracy_df.set_index('topic_no', inplace = True)
accuracy_df.sort_values(by = 'accuracy_score', ascending = False)

,human_topic_desc,accuracy_score
topic_no,,
11,[animal_welfare],1.0
41,[personal_habits.smoking],1.0
35,[diversity_inclusion.sexuality],1.0
28,"[personal_hygiene, housing_facilities]",1.0
29,[beverages],1.0
...,...,...
45,[social_disputes],0.0
44,[arabic_language],0.0
38,[safety.home_safety],0.0


In [64]:
pd.set_option('display.max_rows', None)
accuracy_df

,human_topic_desc,accuracy_score
topic_no,,
-2,[unrelated_theme],0.000000
-1,[undefined],0.539171
0,[reddit_usage],0.361111
1,[uae_regions],0.600000
3,"[diversity_inclusion.race/nationality, diversi...",0.121951
4,[dining_service.food],0.195122
5,[infrastructure.landmarks],0.250000
6,[weather_conditions],0.700000
7,[dining_and_delivery_services],0.277778


In [58]:
accuracy_df[accuracy_df.accuracy_score >=0.5]

,human_topic_desc,accuracy_score
topic_no,,
-1,[undefined],0.539171
1,[uae_regions],0.600000
6,[weather_conditions],0.700000
8,"[taxes_levies, tipping_culture]",0.500000
11,[animal_welfare],1.000000
15,[infrastructure.mobile_communication_internet],0.687500
18,[immigration],0.583333
19,[vehicles],0.545455
20,[social relationships.family],0.500000


In [ ]:
#confusion matrix
y_true = topic_df['human_label']#model_labelled_data['human_label']
y_pred = topic_df ['topic_no'] #model_labelled_data['topic_no']
labels = sorted(list(set(y_true)))
cm = confusion_matrix(y_true = y_true, y_pred = y_pred)
print (cm)

<h2> PRECISION, RECALL, F1-SCORE </h2>

In [59]:
#accuracy score
y_true = model_labelled_data.human_topic_no
y_pred = model_labelled_data.model_topic_no
#labels = list(model_labelled_data['human_topic_no'].unique())
clr = classification_report(y_true, y_pred,zero_division = 0.0, output_dict = True)

In [60]:
clr_df = pd.DataFrame(clr).T
clr_df

,precision,recall,f1-score,support
-2,0.000000,0.000000,0.000000,25.000000
-1,0.393939,0.539171,0.455253,434.000000
0,0.175676,0.361111,0.236364,36.000000
1,0.071429,0.600000,0.127660,10.000000
2,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...
77,0.000000,0.000000,0.000000,12.000000
78,0.000000,0.000000,0.000000,1.000000
accuracy,0.369167,0.369167,0.369167,0.369167
macro avg,0.256614,0.265797,0.234187,1200.000000
